In [1]:
## import libraries for generating visualizations and confusion matrix after loading hf models
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import os
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

/home/hice1/nparikh44/scratch/miniconda/envs/simcse-h100/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## helper function to plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix')
    plt.show()
    
def load_model_and_tokenizer(model_name):
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

def predict(model, tokenizer, texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    return predictions.numpy()

In [3]:
print("Loading dataset...")
ds = load_dataset("lelouch0204/cleaned_allsides_v2.csv")
df = ds['train'].to_pandas()
print(f"Loaded: {len(df)} samples")

# Map bias labels (CORRECTED)
bias_mapping = {
    'left': 4,
    'lean left': 3,
    'center': 2,
    'lean right': 1,
    'right': 0
}
df['bias_label'] = df['Bias'].str.lower().str.strip().map(bias_mapping)
df = df.dropna(subset=['bias_label', 'Text'])
df['bias_label'] = df['bias_label'].astype(int)

# Use existing clean_text or Text
df['text_input'] = df['clean_text'].fillna(df['Text'])
df = df[df['text_input'].str.len() > 50]

print(f"Final dataset: {len(df)} samples")
print("Bias distribution:", df['bias_label'].value_counts().sort_index().to_dict())

# Train-test split
X_text = df['text_input'].values
y = df['bias_label'].values
X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Train: {len(X_train_text)}, Test: {len(X_test_text)}")

Loading dataset...
Loaded: 6209 samples
Final dataset: 6202 samples
Bias distribution: {0: 1273, 1: 549, 2: 644, 3: 776, 4: 2960}
Train: 4961, Test: 1241


## SimCSE fine-tuned

In [ ]:
MODEL_PATH = os.path.expanduser("~/scratch/experiments/allsides_finetuned_simcse_roberta_base/_10_5_epochs_updated_train_test_len_256/")

model, tokenizer = load_model_and_tokenizer(MODEL_PATH)
preds = predict(model, tokenizer, X_test_text.tolist())